<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/main/notebooks/Huggingface_QA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2: Evaluate Dataset

In [1]:
import json
import time
import google.generativeai as genai
from google.colab import userdata
import requests

# API setup
key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=key)
ai_model = genai.GenerativeModel('gemini-1.5-flash')

# Read dataset file
url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Retrieved file: qa_dataset.json")
else:
    print("Error while parsing a file: ", response.status_code)

Retrieved file: qa_dataset.json


In [2]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [3]:
def predict_answers(data):
    """
    Predict the answer for each option in the JSON data.
    Printing only incorrectly predicted answers.
    """
    print("[INFO] Printing only incorrectly predicted answers.")
    correct_count = 0
    total_count = 0

    for item in data:
        predictions = list()

        for option in item['possible_answers']:
            exact_match = option.strip().lower() in item['answer_text'].strip().lower()
            if exact_match:
              predictions.append((option, 0.9)) # 90 % sure its the correct answer
            else:
              result = qa_pipeline(question=item['question'], context=f"{item['answer_text']} {option}")
              predictions.append((option, result['score']))

        predicted_option = max(predictions, key=lambda x: x[1])[0]

        if predicted_option == item['intended_answer']:
            correct_count += 1
        else:
            print(f"Text: {item['answer_text']}")
            print(f"Correct: {item['intended_answer']}, Predicted: {predicted_option}\n")
        total_count += 1

    accuracy = correct_count / total_count if total_count > 0 else 0
    return accuracy

In [4]:
accuracy = predict_answers(data)
print(f"Accuracy: {accuracy * 100:.2f} %")

[INFO] Printing only incorrectly predicted answers.
Text: Oh yeah, I definitely work with wholesalers and distributors, they're a big part of my business.
Correct: Wholesaler, Distributor, Predicted: Consultant, Planner, Architect

Text: Absolutely, my customer group consists of wholesalers and distributors – that's the heart of my operations.
Correct: Wholesaler, Distributor, Predicted: End User

Text: Oh, the customer group?  That'd be consultants, planners, and architects, mostly.
Correct: Consultant, Planner, Architect, Predicted: Wholesaler, Distributor

Text: In terms of client base, we're focused on architects, planners, and consultants for this particular initiative.
Correct: Consultant, Planner, Architect, Predicted: Wholesaler, Distributor

Text: I work in aerospace – it's quite exciting, you know.
Correct: Aerospace, Predicted: Government

Text: My work is all within the government, yeah.
Correct: Government, Predicted: Industrial

Text: I'm knee-deep in the world of automot

## Interesting findings to document later:
*   Prediction of names very bad, because no deeper meaning --> fixed by checking for exact matches
  * Maybe implement name interpreter later?
*   Numerical values (size of company) prediction very bad --> will be fixed by Number module
  * Wobei eigentlich fällt mir grad ein, dass das ja hier als Single Choice ist
  * Dann müssen wa das wohl doch anders machen hehe
  * Das ist wohl ein Problem für morgen (oder so)

